In [7]:
import os
import shutil
from sklearn.model_selection import train_test_split

In [8]:
# Directories
input_dir = 'yoga_combinedd/'  # Raw dataset folder
output_dir = 'yoga_combined/'     # Folder to save the split dataset

In [9]:
# Define pose categories
poses = ['chair', 'cobra', 'downdog', 'goddess', 'tree', 'warrior']


In [10]:
# Create train, valid, test directories
for split in ['train', 'valid', 'test']:
    for pose in poses:
        os.makedirs(os.path.join(output_dir, split, pose), exist_ok=True)

In [16]:
# Function to split and copy images to train, valid, test
def split_data(pose_name):
    pose_path = os.path.join(input_dir, pose_name)
    images = os.listdir(pose_path)  # List the image files in the pose directory
    
    # Ensure only image files are considered
    images = [img for img in images if img.endswith('.jpg') or img.endswith('.png')]

    # Split images: 70% for train, 15% for valid, 15% for test
    train, temp = train_test_split(images, test_size=0.3, random_state=42)
    valid, test = train_test_split(temp, test_size=0.5, random_state=42)
    
    # Copy images to respective directories
    for split, data in zip(['train', 'valid', 'test'], [train, valid, test]):
        for img in data:
            shutil.copy(os.path.join(pose_path, img), os.path.join(output_dir, split, pose_name))


In [17]:
# Process each pose
for pose_name in poses:
    split_data(pose_name)

print("Dataset split completed.")


Dataset split completed.


In [19]:
import os
from PIL import Image

# Function to check if images are valid
def check_image_files(directory):
    for subdir, dirs, files in os.walk(directory):
        for file in files:
            file_path = os.path.join(subdir, file)
            if file.lower().endswith(('.jpg', '.jpeg', '.png', '.bmp')):
                try:
                    with Image.open(file_path) as img:
                        img.verify()  # Verifies if the image is valid
                    print(f"Valid image: {file_path}")
                except Exception as e:
                    print(f"Corrupted or invalid image: {file_path}, Error: {e}")
                    # Optionally, remove the corrupt image
                    # os.remove(file_path)

# Call this function on your dataset directory
check_image_files('yoga_combined/train')


Valid image: yoga_combined/train\chair\1.jpg
Valid image: yoga_combined/train\chair\10.jpg
Valid image: yoga_combined/train\chair\103.jpg
Valid image: yoga_combined/train\chair\106.jpg
Valid image: yoga_combined/train\chair\108.jpg
Valid image: yoga_combined/train\chair\113.jpg
Valid image: yoga_combined/train\chair\114.jpg
Valid image: yoga_combined/train\chair\117.jpg
Valid image: yoga_combined/train\chair\119.jpg
Valid image: yoga_combined/train\chair\12.jpg
Valid image: yoga_combined/train\chair\120.jpg
Valid image: yoga_combined/train\chair\121.jpg
Valid image: yoga_combined/train\chair\123.jpg
Valid image: yoga_combined/train\chair\129.jpg
Valid image: yoga_combined/train\chair\134.jpg
Valid image: yoga_combined/train\chair\135.jpg
Valid image: yoga_combined/train\chair\149.jpg
Valid image: yoga_combined/train\chair\152.jpg
Valid image: yoga_combined/train\chair\164.jpg
Valid image: yoga_combined/train\chair\166.jpg
Valid image: yoga_combined/train\chair\167.jpg
Valid image: yoga

In [21]:
import tensorflow as tf
import os

def preprocess_image(file_path):
    try:
        img = tf.io.read_file(file_path)
        img = tf.image.decode_jpeg(img, channels=3)  # Decode the JPEG
        img = tf.image.resize(img, [224, 224])  # Resize to target size
        img = img / 255.0  # Normalize to [0, 1]
        return img
    except Exception as e:
        print(f"Error processing image {file_path}: {e}")
        return None  # Return None for invalid images

# Test this on a sample image
test_image_path = 'yoga_combined/train/chair/1.jpg'
processed_image = preprocess_image(test_image_path)


In [22]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

datagen = ImageDataGenerator(
    rotation_range=30,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

In [25]:
import os
from PIL import Image

def validate_and_clean_images(directory):
    """
    Validate and clean JPEG images in the given directory.
    """
    for subdir, _, files in os.walk(directory):
        for file in files:
            if file.lower().endswith('.jpg'):
                file_path = os.path.join(subdir, file)
                try:
                    with Image.open(file_path) as img:
                        img.verify()  # Validate the image
                except Exception as e:
                    print(f"Invalid image: {file_path}, Error: {e}")
                    os.remove(file_path)  # Remove the corrupted file
                    print(f"Removed corrupted file: {file_path}")

# Apply to all dataset folders
validate_and_clean_images('yoga_combined/train')
validate_and_clean_images('yoga_combined/valid')
validate_and_clean_images('yoga_combined/test')


In [26]:
import tensorflow as tf

def prepare_data(input_dir, img_height=224, img_width=224, batch_size=32):
    """
    Prepares train and validation datasets.
    """
    train_ds = tf.keras.utils.image_dataset_from_directory(
        os.path.join(input_dir, 'train'),
        image_size=(img_height, img_width),
        batch_size=batch_size
    )
    valid_ds = tf.keras.utils.image_dataset_from_directory(
        os.path.join(input_dir, 'valid'),
        image_size=(img_height, img_width),
        batch_size=batch_size
    )

    # Normalize images to range [0, 1]
    normalization_layer = tf.keras.layers.Rescaling(1./255)
    train_ds = train_ds.map(lambda x, y: (normalization_layer(x), y))
    valid_ds = valid_ds.map(lambda x, y: (normalization_layer(x), y))

    return train_ds, valid_ds


In [27]:
def safe_train(model, train_ds, valid_ds, epochs=10):
    try:
        history = model.fit(
            train_ds,
            validation_data=valid_ds,
            epochs=epochs
        )
        return history
    except tf.errors.InvalidArgumentError as e:
        print(f"Training failed due to an error: {e}")
        return None
